In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [5]:
df_train.columns

Index(['Unnamed: 0', 'NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA',
       'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
       'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL',
       ...
       'Q041', 'Q042', 'Q043', 'Q044', 'Q045', 'Q046', 'Q047', 'Q048', 'Q049',
       'Q050'],
      dtype='object', length=167)

In [6]:
df_train.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,...,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,...,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,...,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,...,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,...,NaN,A,A,A,A,A,A,A,A,A


In [7]:
df_train.fillna(0,inplace=True)
df_test.fillna(0,inplace=True)
labelencoder = LabelEncoder()
df_train[['CO_PROVA_CN']] = labelencoder.fit_transform(df_train[['CO_PROVA_CN']])
df_train[['CO_PROVA_CH']] = labelencoder.fit_transform(df_train[['CO_PROVA_CH']])
df_train[['CO_PROVA_LC']] = labelencoder.fit_transform(df_train[['CO_PROVA_LC']])

df_test[['CO_PROVA_CN']] = labelencoder.fit_transform(df_test[['CO_PROVA_CN']])
df_test[['CO_PROVA_CH']] = labelencoder.fit_transform(df_test[['CO_PROVA_CH']])
df_test[['CO_PROVA_LC']] = labelencoder.fit_transform(df_test[['CO_PROVA_LC']])

In [8]:
df_notas = df_train[['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO','TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC']]

In [9]:
corr = df_notas.corr()
corr.style.background_gradient(cmap='coolwarm')

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC
NU_NOTA_CN,1,0.969323,0.920917,0.907039,0.85781,0.950512,0.950512,0.910132
NU_NOTA_CH,0.969323,1,0.934704,0.903608,0.871199,0.956152,0.956152,0.917
NU_NOTA_LC,0.920917,0.934704,1,0.943954,0.904795,0.913254,0.913254,0.964224
NU_NOTA_MT,0.907039,0.903608,0.943954,1,0.875929,0.874764,0.874764,0.923641
NU_NOTA_REDACAO,0.85781,0.871199,0.904795,0.875929,1,0.821287,0.821287,0.865595
TP_PRESENCA_CN,0.950512,0.956152,0.913254,0.874764,0.821287,1,1,0.943361
TP_PRESENCA_CH,0.950512,0.956152,0.913254,0.874764,0.821287,1,1,0.943361
TP_PRESENCA_LC,0.910132,0.917,0.964224,0.923641,0.865595,0.943361,0.943361,1


In [10]:
selected_columns = df_notas.drop(columns=['NU_NOTA_MT']).columns
import statsmodels.api as sm
def backwardElimination(x, Y, sl, columns):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(Y,x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
                    columns = np.delete(columns, j)
                    
    regressor_OLS.summary()
    return x, columns

In [11]:
SL = 0.05
data_modeled, selected_columns = backwardElimination(df_notas.drop(columns=['NU_NOTA_MT']).values, df_notas.iloc[:,3].values, SL, selected_columns)

In [12]:
result = pd.DataFrame()
result['NU_NOTA_MT'] = df_notas.iloc[:,3]

In [13]:
data = pd.DataFrame(data = data_modeled, columns = selected_columns)
selected_columns

Index(['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC'],
      dtype='object')

In [14]:
x_train, x_test, y_train, y_test = train_test_split(data.values, result.values, test_size = 0.2)

In [15]:
reg = LinearRegression().fit(x_train, y_train)
Y = reg.predict(df_test[selected_columns])
test_result = pd.DataFrame(data=df_test['NU_INSCRICAO'],
              columns=['NU_INSCRICAO'])
test_result['NU_NOTA_MT'] = Y[0:,0]
test_result.loc[test_result['NU_NOTA_MT'] < 0, 'NU_NOTA_MT'] = 0
test_result

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,431.517040
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,454.888482
2,b38a03232f43b11c9d0788abaf060f7366053b6d,571.660171
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,0.000000
4,715494628a50142ce8cb17191cfe6d0f3cae0934,541.813482
5,e656d6bad65c93fb2880f1eba5037008c8e75774,469.893324
6,465cd2a6907fb37d9d8ad3c065f0e2dabdba9b13,538.006205
7,11539e86171bf07d3a36f09377d7f54ebcc8406a,427.601328
8,043c544a2104aa8a9849f1a703a08d37a2f16839,482.676149
9,76ba050e64ad100b856f0eaabd8f539d5c7dd185,465.134872


In [16]:
test_result.to_csv('answer.csv', index=False)